<a href="https://colab.research.google.com/github/narcisonascimento/pyspark-estudos/blob/main/pyspark_estudos_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark Estudos <br>
Fundamentos do Pyspark <br>
Professor: Renan Marques Rodrigues <br>
Youtube: DataDev Academy | https://www.youtube.com/@datadevacademy

# Instalando bibliotecas

In [2]:
# instalando pyspark
# !pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b46d9a82d0d2a21b3809b4f49daed97dfa789527698d16e1de381c257dec929b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
# pip freeze

# Importando Bibliotecas

In [3]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# os.environ['PYSPARK_PYTHON'] = sys.executable
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

Criar / iniciar Sessão PySpark

In [4]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('pyspark-estudos')
    .getOrCreate()
)

Criar DF / ler arquivo

Verificar tipos de colunas

In [6]:
df = spark.read.csv('/content/wc2018-players.csv', header=True, inferSchema=True)
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



In [7]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



Verificar dados nulos

In [8]:
# transformando em Pandas e efetuando a soma. Aqui só funciona com poucas linhas no dataset
df.toPandas().isna().sum()

Team                 0
#                    0
Pos.                 0
FIFA Popular Name    0
Birth Date           0
Shirt Name           0
Club                 0
Height               0
Weight               0
dtype: int64

In [10]:
# verificando as colunas existentes
df.columns

['Team',
 '#',
 'Pos.',
 'FIFA Popular Name',
 'Birth Date',
 'Shirt Name',
 'Club',
 'Height',
 'Weight']

In [13]:
# No PySpark deve ser feito um looping for, esse looping demora dependendo do tamanho do dataset

for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

Team 0
# 0


AnalysisException: Syntax error in attribute name: Pos..

Renomear colunas <br>
PySpark não aceita pontos no nome e também deve ter atenção com espaço entre palavras.

In [14]:
df = df.withColumnRenamed('Team', 'selecao')\
.withColumnRenamed('#', 'numero')\
.withColumnRenamed('Pos.', 'posicao')\
.withColumnRenamed('FIFA Popular Name', 'nome_fifa')\
.withColumnRenamed('Birth Date', 'data_nascimento')\
.withColumnRenamed('Shirt Name', 'nome_camisa')\
.withColumnRenamed('Club', 'time')\
.withColumnRenamed('Height', 'altura')\
.withColumnRenamed('Weight', 'peso')

In [15]:
df.columns

['selecao',
 'numero',
 'posicao',
 'nome_fifa',
 'data_nascimento',
 'nome_camisa',
 'time',
 'altura',
 'peso']

In [16]:
# após as devidas correções é possível efetuar o looping no dataset

for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

selecao 0
numero 0
posicao 0
nome_fifa 0
data_nascimento 0
nome_camisa 0
time 0
altura 0
peso 0


Selecionar colunas

In [17]:
# diferente do Pandas o PySpark altera o nome da coluna original quando escrita de forma diferente (caps lock etc).
df.select('selecao', 'nome_fifa').show(5)

+---------+------------------+
|  selecao|         nome_fifa|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [19]:
df.select('SELECAO', 'NOME_FIFA').show(5)

+---------+------------------+
|  SELECAO|         NOME_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



Pode selecionar colunas com o 'col'

In [24]:
df.select(col('selecao'), col('nome_camisa'), col('altura')).show(5)

+---------+-----------+------+
|  selecao|nome_camisa|altura|
+---------+-----------+------+
|Argentina| TAGLIAFICO|   169|
|Argentina|      PAVÓN|   169|
|Argentina|    LANZINI|   167|
|Argentina|     SALVIO|   167|
|Argentina|      MESSI|   170|
+---------+-----------+------+
only showing top 5 rows



ou pode selecioar com o dataframe['nome_coluna']

In [25]:
df.select(df['selecao'], df['nome_camisa'], df['altura']).show(5)

+---------+-----------+------+
|  selecao|nome_camisa|altura|
+---------+-----------+------+
|Argentina| TAGLIAFICO|   169|
|Argentina|      PAVÓN|   169|
|Argentina|    LANZINI|   167|
|Argentina|     SALVIO|   167|
|Argentina|      MESSI|   170|
+---------+-----------+------+
only showing top 5 rows



Selecionar Colunas com ALIAS

In [26]:
df.select(col('selecao').alias('time')).show(5)

+---------+
|     time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [27]:
# utilizando o split para separar as colunas
df.select('selecao nome_fifa altura'.split()).show(5)

+---------+------------------+------+
|  selecao|         nome_fifa|altura|
+---------+------------------+------+
|Argentina|TAGLIAFICO Nicolas|   169|
|Argentina|    PAVON Cristian|   169|
|Argentina|    LANZINI Manuel|   167|
|Argentina|    SALVIO Eduardo|   167|
|Argentina|      MESSI Lionel|   170|
+---------+------------------+------+
only showing top 5 rows



Organizar o Select

In [28]:
df.select('nome_camisa', 'altura', 'peso').show(5)

+-----------+------+----+
|nome_camisa|altura|peso|
+-----------+------+----+
| TAGLIAFICO|   169|  65|
|      PAVÓN|   169|  65|
|    LANZINI|   167|  66|
|     SALVIO|   167|  69|
|      MESSI|   170|  72|
+-----------+------+----+
only showing top 5 rows



Filtrar o dataframe

In [29]:
df.filter('selecao = "Brazil"').show(10)

+-------+------+-------+-----------------+---------------+-----------+--------------------+------+----+
|selecao|numero|posicao|        nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|
+-------+------+-------+-----------------+---------------+-----------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|     05.03.1993|       FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|     13.01.1988|     TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|     04.05.1985|FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|     11.06.1989|     FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|           NEYMAR|     05.02.1992|  NEYMAR JR|Paris Saint-Germa...|   175|  68|
| Brazil|    11|     MF|PHILIPPE COUTINHO|     12.06.1992|P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
| Brazil|     7|     FW|    DOUGLAS COSTA|     14.09.1990|   D. 

In [35]:
# caso o nome da coluna possua espaço, não pode fazer o filter direto como abaixo, deve chamar a função col
df.filter('nome_camisa == "FRED"').show()

+-------+------+-------+---------+---------------+-----------+--------------------+------+----+
|selecao|numero|posicao|nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|
+-------+------+-------+---------+---------------+-----------+--------------------+------+----+
| Brazil|    18|     MF|     FRED|     05.03.1993|       FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+---------+---------------+-----------+--------------------+------+----+



In [39]:
df.filter(col('nome_camisa') == 'FRED').show()

+-------+------+-------+---------+---------------+-----------+--------------------+------+----+
|selecao|numero|posicao|nome_fifa|data_nascimento|nome_camisa|                time|altura|peso|
+-------+------+-------+---------+---------------+-----------+--------------------+------+----+
| Brazil|    18|     MF|     FRED|     05.03.1993|       FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+---------+---------------+-----------+--------------------+------+----+



Filtrar DF com 2 condições (and/&)